In [1]:
import pandas as pd
import numpy as np
import string
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
!pip install -U textblob
!pip install nltk 
!pip install catboost

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

Requirement already up-to-date: textblob in /usr/local/lib/python3.6/dist-packages (0.15.3)
     |████████████████████████████████| 64.8MB 60kB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1Pk5MK9Hs_kMUT9NotGnOKE0NPra-39YU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

train = pd.read_csv('train.csv')

In [5]:
id='1GsTM9oLtIV8-Da_fDOFWsQYMpgQ8GOYJ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test.csv')

test = pd.read_csv('test.csv')

In [15]:
id='1slJiG-M_oawvkzaqWzZkSmVxxZor6mKQ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_features.csv')

id='1TmL2AY_yymiV8zXVvmbqXsIZoexR9sLs'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_features.csv')
train_features = pd.read_csv('train_features.csv')
test_features = pd.read_csv('test_features.csv')

In [16]:
#Pasos NLP:
#1: FILTRADO DE DATOS
#2: TOKENIZACION
#3: STOPWORDS
#4: LEMATIZACION

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer() 

def nlp_text(text):
    #filtrado de signos de puntuacion
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    #tokenizacion
    tokens = re.split('\W+', text)
    #lematizacion
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

PARA TRAIN

In [39]:
tf_idf = TfidfVectorizer(analyzer=nlp_text, max_features=5000)
tf_idf = tf_idf.fit_transform(train.text)
matrix_tf_idf = tf_idf.todense()
matrix_tf_idf.shape

(7613, 5000)

In [27]:
df_matrix_tf_idf = pd.DataFrame(matrix_tf_idf)

In [18]:
numeric_features = train_features[['cant_stop_words', 'prom_long_palabra', 'cant_puntuacion', 'cant_apariciones_keyword', 'cant_numeros', 'cant_mayusculas', 'cant_vocales']]
numeric_features

,cant_stop_words,prom_long_palabra,cant_puntuacion,cant_apariciones_keyword,cant_numeros,cant_mayusculas,cant_vocales
0,6,4.384615,1,61,0,10,25
1,0,4.571429,1,61,0,5,13
2,11,5.090909,3,61,0,2,45
3,1,7.125000,2,61,5,1,24
4,7,4.500000,2,61,0,3,25
...,...,...,...,...,...,...,...
7608,2,6.636364,5,61,1,7,20
7609,9,5.300000,5,61,0,6,39
7610,2,7.250000,11,61,9,10,12
7611,5,6.263158,5,61,0,4,49


In [29]:
matriz_tfidf_train = pd.concat([df_matrix_tf_idf, numeric_features], axis=1, sort=False)

In [30]:
matriz_tfidf_train.to_csv('features_train_tf_idf.csv', index=False)

In [31]:
matriz_tfidf_train.shape

(7613, 5007)

In [32]:
from google.colab import files
files.download('features_train_tf_idf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

PARA TEST

In [34]:
tf_idf = TfidfVectorizer(analyzer=nlp_text, max_features=5000)
tf_idf_test = tf_idf.fit_transform(test.text)
matrix_tf_idf_test = tf_idf_test.todense()

df_matrix_tf_idf_test = pd.DataFrame(matrix_tf_idf_test)

numeric_features_test = test_features[['cant_stop_words', 'prom_long_palabra', 'cant_puntuacion', 'cant_apariciones_keyword', 'cant_numeros', 'cant_mayusculas', 'cant_vocales']]

matriz_tfidf_test = pd.concat([df_matrix_tf_idf_test, numeric_features_test], axis=1, sort=False)
matriz_tfidf_test.to_csv('features_test_tf_idf.csv', index=False)

In [36]:
matriz_tfidf_test.shape

(3263, 5007)

In [47]:
from google.colab import files
files.download('features_test_tf_idf.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>